In [1]:
import lazypredict

# Read dataframe

In [7]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/balanced_dataset2.csv"
df = pd.read_csv(filepath)
df

,page_id,page_title,view_count,protection_status,page_length,edit_count,page_watchers,page_watchers_recent_edits,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
0,50537912,2016_Stanley_Cup_finals,1015,sysop,69,2,25,50,1,50537912,0,0,0,2
1,46633273,2015_Stanley_Cup_finals,1238,sysop,69,2,25,50,1,46633273,0,0,0,2
2,50537916,2016_stanley_cup_finals,2007,sysop,69,2,25,50,2,50537916,0,0,0,2
3,39323133,2013_stanley_cup_finals,2048,sysop,69,2,25,50,2,39323133,0,0,0,2
4,42486165,2014_Stanley_Cup_finals,851,sysop,69,3,25,50,2,42486165,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,3284182,List of spells in Harry Potter,2208118,sysop,0,0,416,12,0,0,0,5,2,15
829,0,1848â€“49_massacres_in_Transylvania,1,sysop,0,0,25,50,0,0,0,5,2,15
830,49078342,Tony Krolis,558,unprotected,0,0,25,50,0,0,0,5,2,15
831,25198426,Mysandbox,1067,sysop,0,0,25,50,0,0,0,5,2,15


# Prepare the data (pre-processing)

In [8]:
import numpy as np

features = df

# Labels are the values we want to predict
labels = np.array(df['protection_status'])

# Remove the labels from the features
# axis 1 refers to the columns
features = features.drop('protection_status', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('page_id', axis=1)
features = features.drop('page_id_scrapped', axis=1)

#Mapping for encoding
# Page Watchers column: 25 => "Fewer than 30 watchers"
# page_watchers_recent_edits column: "50 (average of this column) => "There may or may not be a watching user visiting recent edits"
# All Columns => 0: NaN   


#Encode string to float
features.loc[features['page_watchers'] == "Fewer than 30 watchers", 'page_watchers'] = 25
features.loc[features['page_watchers_recent_edits'] == "There may or may not be a watching user visiting recent edits", 'page_watchers_recent_edits'] = 50
features = features.fillna(0)

#Convert String to Floats
features['page_length'] = features['page_length'].astype(float)
features['edit_count'] = features['edit_count'].astype(float)
features['page_watchers'] = features['page_watchers'].astype(float)
features['page_watchers_recent_edits'] = features['page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

## Encode categorical values

In [9]:
#Label encoding for protection_status column

# 0 => unprotected
# 1 => autoconfirmed
# 2 => extendedconfirmed
# 3 => sysop
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)   

# Model training and results

In [10]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.10, random_state = 25)

In [11]:
X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 11.56it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
XGBClassifier                      0.73               0.71    None      0.72   
LGBMClassifier                     0.73               0.71    None      0.72   
ExtraTreesClassifier               0.68               0.66    None      0.67   
RandomForestClassifier             0.64               0.62    None      0.63   
DecisionTreeClassifier             0.62               0.61    None      0.62   
BaggingClassifier                  0.63               0.60    None      0.62   
AdaBoostClassifier                 0.60               0.57    None      0.58   
ExtraTreeClassifier                0.52               0.52    None      0.52   
LogisticRegression                 0.55               0.51    None      0.51   
SVC                                0.54               0.50    None      0.50   
LinearDiscriminantAnalysis         0.54 

# XGBoost Model

In [12]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

#Train
modelXG = XGBClassifier()
modelXG.fit(X_train, y_train)

#Predict
y_pred = modelXG.predict(X_test)
predictions = [round(value) for value in y_pred]


#Evaluate
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 72.62%


## Save the model

In [13]:
import pickle
# save the model to disk
filename = 'models/model_72.sav'
pickle.dump(model, open(filename, 'wb'))
 


NameError: name 'model' is not defined